<a href="https://colab.research.google.com/github/jmbaek/DNABERT/blob/master/DNABERT_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jerryji1993/DNABERT

Cloning into 'DNABERT'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (766/766), done.
remote: Compressing objects: 100% (529/529), done.
remote: Total 766 (delta 400), reused 552 (delta 214), pack-reused 0
Receiving objects: 100% (766/766), 11.78 MiB | 3.83 MiB/s, done.
Resolving deltas: 100% (400/400), done.


In [2]:
%cd /content/DNABERT
!python -m pip install --editable .
!cd examples
!python3 -m pip install -r requirements.txt

/content/DNABERT
Obtaining file:///content/DNABERT
     |████████████████████████████████| 5.7 MB 3.2 MB/s 
     |████████████████████████████████| 132 kB 74.9 MB/s 
     |████████████████████████████████| 1.2 MB 57.7 MB/s 
     |████████████████████████████████| 895 kB 82.8 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 8.6 MB 73.1 MB/s 
     |████████████████████████████████| 138 kB 92.5 MB/s 
     |████████████████████████████████| 127 kB 88.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Running setup.py develop for transformers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: Could not o

In [3]:
!pwd
!mkdir 6-new-12w-0
!mkdir finetuned

## upload model files and move to the model_path

/content/DNABERT


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
import tensorflow
import json

with open('/content/DNABERT/6-new-12w-0/config.json', "rb") as json_data:
    print(json_data)
    json_config = json.load(json_data)

new_model = tensorflow.keras.models.model_from_json(json_config)

new_model.summary()

<_io.BufferedReader name='/content/DNABERT/6-new-12w-0/config.json'>


JSONDecodeError: ignored

In [8]:
!cd examples

!python /content/DNABERT/examples/run_finetune.py \
    --model_type dna \
    --tokenizer_name=dna6 \
    --model_name_or_path /content/DNABERT/6-new-12w-0 \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir /content/DNABERT/examples/sample_data/ft/6 \
    --max_seq_length 100 \
    --per_gpu_eval_batch_size=32   \
    --per_gpu_train_batch_size=32   \
    --learning_rate 2e-4 \
    --num_train_epochs 5.0 \
    --output_dir /content/DNABERT/finetuned/6 \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8

04/05/2022 02:12:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/05/2022 02:12:33 - INFO - transformers.configuration_utils -   loading configuration file /content/DNABERT/6-new-12w-0/config.json
04/05/2022 02:12:33 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": "dnaprom",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,


## Prediction

In [ ]:
!mkdir prediction

!python /content/DNABERT/examples/run_finetune.py \
    --model_type dna \
    --tokenizer_name=dna6 \
    --model_name_or_path /content/DNABERT/finetuned/6 \
    --task_name dnaprom \
    --do_predict \
    --data_dir /content/DNABERT/examples/sample_data/ft/6  \
    --max_seq_length 75 \
    --per_gpu_pred_batch_size=128   \
    --output_dir /content/DNABERT/finetuned/6 \
    --predict_dir /content/DNABERT/prediction \
    --n_process 48